# Getting Started with the OpenAI Agents SDK

This notebook will guide you through the basics of using the [OpenAI Agents SDK](https://github.com/openai/agents-sdk) with Amazon Bedrock. We'll cover how to set up your environment, create an agent, and run a simple example.

## Prerequisites
Before you begin, ensure you have the following:
- Python installed
- An AWS account with access to Amazon Bedrock
- The `openai-agents-sdk` Python package installed

You can install the `openai-agents` package using pip:

In [1]:
%pip install openai-agents litellm boto3 --quiet --upgrade

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from __future__ import annotations
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.tool import FunctionTool

set_tracing_disabled(disabled=True)

## Converting an OpenAI Tool to a Bedrock Tool

The `convert_openai_tool_to_bedrock_tool` function converts an OpenAI tool to a Bedrock tool. This is useful when you want to use an existing OpenAI tool with Amazon Bedrock.

In [3]:
def convert_openai_tool_to_bedrock_tool(tool: dict) -> FunctionTool:
    """Converts an OpenAI tool to a Bedrock tool."""
    return FunctionTool(
        name=tool["name"],
        description=tool["description"],
        params_json_schema={
            "type": "object",
            "properties": { k: v for k, v in tool["params_json_schema"]["properties"].items() },
            "required": tool["params_json_schema"].get("required", []),
        },
        on_invoke_tool=tool["on_invoke_tool"],
    )

## Creating a Simple Tool

Let's create a simple tool that gets the weather for a given city.

In [4]:
@function_tool
def get_weather(city: str):
    """Get the weather for a given city."""
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is sunny."

## Creating an Agent

Now, let's create an agent that uses the `get_weather` tool.

In [10]:
%autoawait asyncio

agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model="litellm/bedrock/us.amazon.nova-pro-v1:0",
    tools=[convert_openai_tool_to_bedrock_tool(get_weather.__dict__)],
)

result = await Runner.run(agent, "What's the weather in Tokyo today, April 22 2025?")
print(result.final_output)

/opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/typing.py:1217: RuntimeWarning: coroutine 'main' was never awaited
  super().__setattr__(attr, val)


[debug] getting weather for Tokyo
Sunny skies in Tokyo,
April 22, 2025 haiku.
